In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.errors import InvalidArgumentError
import tensorflow as tf

# Tensorflow GPU settings
# gpu_options = tf.GPUOptions(allow_growth=True)#per_process_gpu_memory_fraction=0.5)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import shuffle

from scipy import interpolate

from atlasify import atlasify
import pickle

2022-01-29 05:16:26.312851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-29 05:16:27.180833: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-29 05:16:27.181870: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-29 05:16:27.188771: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-01-29 05:16:27.188881: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-29 05:16:27.190225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-0

In [2]:
sig = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_VVZ_RD.arrow')
sig['is_signal'] = True
# sig_test = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/SIG_2021_11_16_no_iso_TEST.arrow')
bg_full = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_FULLBG_RD.arrow')
bg_full['is_signal'] = False
# bg_test = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/FULLBG_2021_11_16_no_iso_TEST.arrow')

In [3]:
bg_ZZ = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_ZZ_RD.arrow')
bg_Zjets_old = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20211129_iso_e4m1_Zjets_RD.arrow')
bg_Zjets = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_Zjets_RD.arrow')
bg_Zgamma = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_Zgamma_RD.arrow')
bg_WZ = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_WZ_RD.arrow')
bg_tZ = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_tZ_RD.arrow')
bg_tWZ = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_tWZ_RD.arrow')
bg_ttZ = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_ttZ_RD.arrow')
bg_other = pd.read_feather('/home/grabanal/WVZ/gabriel_ML_data/20220117_iso_e4m1_others_RD.arrow')

bg_sources = [bg_ZZ, bg_Zjets, bg_Zgamma, bg_WZ, bg_tZ, bg_tWZ, bg_ttZ, bg_other]

In [4]:
for df in bg_sources:
    print(sum(df.wgt) / sum(bg_full.wgt))

0.9717740665613699
0.008909331116206785
0.0010556883632736617
0.004347097929154949
0.00016711686720929022
0.0025379501654372405
0.005859107932870372
0.006691005792313908


In [9]:
# train_feats = ['Zlep1_dphi', 'Zlep2_dphi', 'Wlep1_dphi', 'Wlep2_dphi', 'pt_1', 'pt_2', 'Wlep2_pt_sqrt']
# train_feats = ['pt_1', 'pt_2', 'pt_3', 'pt_4', 'pt_4l',
#                'Zlep1_dphi', 'Zlep2_dphi', 'Wlep1_dphi', 'Wlep2_dphi', 
#                'Zlep1_phi', 'Zlep2_phi', 'Wlep1_phi', 'Wlep2_phi', 
#                'Zlep1_eta', 'Zlep2_eta', 'Wlep1_eta', 'Wlep2_eta',
#                'Zlep1_pid', 'Zlep2_pid', 'Wlep1_pid', 'Wlep2_pid',
#                'Zlep1_pt', 'Zlep2_pt', 'Wlep1_pt', 'Wlep2_pt',
#                'METSig', 'MET', 'Nlep', 'Njet',
#                'other_mass', 'leptonic_HT', 'total_HT', 'HT', 
#                'SR']

train_feats_raw = sorted([f for f in sig.columns if f not in ['index', 'wgt', 'is_signal', 'Zcand_mass', 'chisq']])

X = pd.concat([sig[train_feats_raw], bg_full[train_feats_raw]], ignore_index=True)

In [10]:
# Normalize inputs for NN training
min_max_scaler = preprocessing.MinMaxScaler()

min_max_scaler.fit(X)

for df in [sig, bg_full] + bg_sources:
    df[train_feats_raw] = min_max_scaler.transform(df[train_feats_raw])

# Signal vs other

other events account for

In [11]:
current_bg = bg_other
sum(current_bg.wgt) / sum(bg_full.wgt) * 100

0.6691005792313908

percent of the total background. The amount relative to signal is

In [12]:
print('Number:', len(current_bg)/len(sig) * 100, '%')
print('Weight:', sum(current_bg.wgt)/sum(sig.wgt) * 100, '%')

Number: 0.5516142608068297 %
Weight: 56.2119517269744 %


Now we can scan over different training setups.

In [13]:
np.asarray(train_feats_raw)

array(['HT', 'MET', 'METSig', 'Njet', 'Nlep', 'SR', 'Wlep1_dphi',
       'Wlep1_eta', 'Wlep1_phi', 'Wlep1_pid', 'Wlep1_pt', 'Wlep2_dphi',
       'Wlep2_eta', 'Wlep2_phi', 'Wlep2_pid', 'Wlep2_pt', 'Zlep1_dphi',
       'Zlep1_eta', 'Zlep1_phi', 'Zlep1_pid', 'Zlep1_pt', 'Zlep2_dphi',
       'Zlep2_eta', 'Zlep2_phi', 'Zlep2_pid', 'Zlep2_pt', 'leptonic_HT',
       'mass_4l', 'other_mass', 'pt_1', 'pt_2', 'pt_3', 'pt_4', 'pt_4l',
       'total_HT'], dtype='<U11')

In [14]:
train_feat_sets = [train_feats_raw, 
                   [f for f in train_feats_raw if f not in ['Wlep1_phi', 'Wlep2_phi', 'Zlep1_phi', 'Zlep2_phi']],
                   [f for f in train_feats_raw if f not in ['MET', 'METSig']],
                   [f for f in train_feats_raw if f not in ['pt_1', 'pt_2', 'pt_3', 'pt_4']],
                   [f for f in train_feats_raw if f not in ['Njet', 'Nlep']]
                  ]

In [ ]:
EPOCHS = 5000
patience = 500
batch_size = 512
num_nodes = 32
dropout = 0.1
learn_rate = 1e-5

for i, train_feats in enumerate(train_feat_sets):
    print('Running with training features:', train_feats)
    # Save training setup
    with open('models/background_id_models/classifier_other_train_feat_test_' + str(i) + '_setup.txt', 
              'w') as file:
        file.write('Epochs: ' + str(EPOCHS) + '\n')
        file.write('Patience: ' + str(patience) + '\n')
        file.write('Learning rate: ' + str(learn_rate) + '\n')
        file.write('Batch size: ' + str(batch_size) + '\n\n')
        file.write('Training features:\n' + '\n'.join(train_feats))
    
    # Generate train and test samples
    sig_train, sig_test = train_test_split(sig[train_feats + ['wgt']], train_size=0.5, random_state=314)
    bg_train, bg_test = train_test_split(current_bg[train_feats + ['wgt']], train_size=0.5, random_state=314)

    n_sig = sum(sig_train.wgt)
    n_bg = sum(bg_train.wgt)

    x_train_sig = sig_train[train_feats]
    x_train_bg = bg_train[train_feats]

    x_train = pd.concat([x_train_sig, x_train_bg])
    y_train = np.concatenate([np.ones(len(sig_train)), np.zeros(len(bg_train))])
    w_train = pd.Series(np.concatenate([(n_sig + n_bg) / n_sig * sig_train['wgt'], 
                                        (n_sig + n_bg) / n_bg * bg_train['wgt']]))

    n_sig_test = sum(sig_test.wgt)
    n_bg_test = sum(bg_test.wgt)

    x_test = pd.concat([sig_test[train_feats], bg_test[train_feats]])
    y_test = np.concatenate([np.ones(len(sig_test)), np.zeros(len(bg_test))])
    w_test = pd.Series(np.concatenate([(n_sig_test + n_bg_test) / n_sig_test * sig_test['wgt'], 
                                       (n_sig_test + n_bg_test) / n_bg_test * bg_test['wgt']]))
    
    # Generate and fit model
    K.clear_session()
    classifier_other = Sequential()
    classifier_other.add(Dense(num_nodes, input_dim=x_train.shape[1], activation='relu')) 
    classifier_other.add(Dropout(dropout))
    classifier_other.add(Dense(num_nodes, activation='relu'))
    classifier_other.add(Dropout(dropout))
    classifier_other.add(Dense(num_nodes, activation='relu'))
    classifier_other.add(Dropout(dropout))
    classifier_other.add(Dense(1, activation='sigmoid'))

    opt = keras.optimizers.Adam(learning_rate=learn_rate)
    classifier_other.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    # Early stopping
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    history = classifier_other.fit(x_train, y_train, epochs=EPOCHS, batch_size=batch_size,
                                 validation_data=(x_test, y_test, w_test), sample_weight=w_train, 
                                 verbose=1, callbacks=[callback], shuffle=True)
    
    # Save model and history
    classifier_other.save('models/background_id_models/classifier_other_train_feat_test_' + str(i))
    with open('models/background_id_models/classifier_other_train_feat_test_' + str(i) + '_history.pkl', 
              'wb') as file_pi:
        pickle.dump(history.history, file_pi)

Running with training features: ['HT', 'MET', 'METSig', 'Njet', 'Nlep', 'SR', 'Wlep1_dphi', 'Wlep1_eta', 'Wlep1_phi', 'Wlep1_pid', 'Wlep1_pt', 'Wlep2_dphi', 'Wlep2_eta', 'Wlep2_phi', 'Wlep2_pid', 'Wlep2_pt', 'Zlep1_dphi', 'Zlep1_eta', 'Zlep1_phi', 'Zlep1_pid', 'Zlep1_pt', 'Zlep2_dphi', 'Zlep2_eta', 'Zlep2_phi', 'Zlep2_pid', 'Zlep2_pt', 'leptonic_HT', 'mass_4l', 'other_mass', 'pt_1', 'pt_2', 'pt_3', 'pt_4', 'pt_4l', 'total_HT']


2022-01-29 05:17:14.888190: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-29 05:17:14.890433: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-01-29 05:17:14.890483: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-29 05:17:14.890500: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-29 05:17:14.890509: I tensorflow/stream_executor/plat

Epoch 1/5000
 1/66 [..............................] - ETA: 33s - loss: 4.1212e-04 - accuracy: 0.7305

2022-01-29 05:17:15.742493: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


66/66 [==============================] - 1s 11ms/step - loss: 5.3406e-04 - accuracy: 0.7052 - val_loss: 6.2115e-04 - val_accuracy: 0.6913
Epoch 2/5000
66/66 [==============================] - 0s 7ms/step - loss: 5.7930e-04 - accuracy: 0.6961 - val_loss: 6.2092e-04 - val_accuracy: 0.6868
Epoch 3/5000
66/66 [==============================] - 0s 7ms/step - loss: 5.5416e-04 - accuracy: 0.6932 - val_loss: 6.2069e-04 - val_accuracy: 0.6827
Epoch 4/5000
66/66 [==============================] - 0s 7ms/step - loss: 5.5405e-04 - accuracy: 0.6848 - val_loss: 6.2048e-04 - val_accuracy: 0.6768
Epoch 5/5000
66/66 [==============================] - 0s 7ms/step - loss: 5.5667e-04 - accuracy: 0.6790 - val_loss: 6.2028e-04 - val_accuracy: 0.6715
Epoch 6/5000
66/66 [==============================] - 0s 7ms/step - loss: 5.7724e-04 - accuracy: 0.6760 - val_loss: 6.2006e-04 - val_accuracy: 0.6643
Epoch 7/5000
66/66 [==============================] - 0s 7ms/step - loss: 5.4680e-04 - accuracy: 0.6734 - val_lo

...

66/66 [==============================] - 0s 7ms/step - loss: 3.2626e-04 - accuracy: 0.8756 - val_loss: 3.7045e-04 - val_accuracy: 0.8911
Epoch 2191/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.2722e-04 - accuracy: 0.8722 - val_loss: 3.7081e-04 - val_accuracy: 0.8926
Epoch 2192/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9979e-04 - accuracy: 0.8775 - val_loss: 3.7094e-04 - val_accuracy: 0.8929
Epoch 2193/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0696e-04 - accuracy: 0.8795 - val_loss: 3.7106e-04 - val_accuracy: 0.8934
Epoch 2194/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0418e-04 - accuracy: 0.8777 - val_loss: 3.7137e-04 - val_accuracy: 0.8944
Epoch 2195/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0687e-04 - accuracy: 0.8780 - val_loss: 3.7130e-04 - val_accuracy: 0.8943
Epoch 2196/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9865e-04 - accuracy

66/66 [==============================] - 0s 7ms/step - loss: 2.8764e-04 - accuracy: 0.8722 - val_loss: 3.7041e-04 - val_accuracy: 0.8905
Epoch 2244/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9027e-04 - accuracy: 0.8736 - val_loss: 3.7049e-04 - val_accuracy: 0.8910
Epoch 2245/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9150e-04 - accuracy: 0.8726 - val_loss: 3.7057e-04 - val_accuracy: 0.8915
Epoch 2246/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.2993e-04 - accuracy: 0.8783 - val_loss: 3.7073e-04 - val_accuracy: 0.8922
Epoch 2247/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.3231e-04 - accuracy: 0.8714 - val_loss: 3.7038e-04 - val_accuracy: 0.8908
Epoch 2248/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.8057e-04 - accuracy: 0.8736 - val_loss: 3.7071e-04 - val_accuracy: 0.8923
Epoch 2249/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0913e-04 - accuracy

66/66 [==============================] - 0s 7ms/step - loss: 2.9641e-04 - accuracy: 0.8732 - val_loss: 3.7016e-04 - val_accuracy: 0.8905
Epoch 2297/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0627e-04 - accuracy: 0.8684 - val_loss: 3.7017e-04 - val_accuracy: 0.8904
Epoch 2298/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.7485e-04 - accuracy: 0.8766 - val_loss: 3.7046e-04 - val_accuracy: 0.8916
Epoch 2299/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0822e-04 - accuracy: 0.8736 - val_loss: 3.7081e-04 - val_accuracy: 0.8935
Epoch 2300/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9733e-04 - accuracy: 0.8755 - val_loss: 3.7133e-04 - val_accuracy: 0.8958
Epoch 2301/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.1634e-04 - accuracy: 0.8793 - val_loss: 3.7141e-04 - val_accuracy: 0.8958
Epoch 2302/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9766e-04 - accuracy

66/66 [==============================] - 0s 7ms/step - loss: 3.0531e-04 - accuracy: 0.8753 - val_loss: 3.7045e-04 - val_accuracy: 0.8907
Epoch 2350/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.7850e-04 - accuracy: 0.8755 - val_loss: 3.7064e-04 - val_accuracy: 0.8916
Epoch 2351/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.1112e-04 - accuracy: 0.8731 - val_loss: 3.7063e-04 - val_accuracy: 0.8917
Epoch 2352/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.0832e-04 - accuracy: 0.8717 - val_loss: 3.7107e-04 - val_accuracy: 0.8935
Epoch 2353/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.8586e-04 - accuracy: 0.8759 - val_loss: 3.7079e-04 - val_accuracy: 0.8925
Epoch 2354/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.3108e-04 - accuracy: 0.8766 - val_loss: 3.7049e-04 - val_accuracy: 0.8906
Epoch 2355/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9516e-04 - accuracy

66/66 [==============================] - 0s 7ms/step - loss: 2.7376e-04 - accuracy: 0.8773 - val_loss: 3.7078e-04 - val_accuracy: 0.8938
Epoch 2403/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.7186e-04 - accuracy: 0.8767 - val_loss: 3.7065e-04 - val_accuracy: 0.8932
Epoch 2404/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.2825e-04 - accuracy: 0.8731 - val_loss: 3.7050e-04 - val_accuracy: 0.8926
Epoch 2405/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.7038e-04 - accuracy: 0.8783 - val_loss: 3.7063e-04 - val_accuracy: 0.8935
Epoch 2406/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.9168e-04 - accuracy: 0.8781 - val_loss: 3.7044e-04 - val_accuracy: 0.8924
Epoch 2407/5000
66/66 [==============================] - 0s 7ms/step - loss: 2.8182e-04 - accuracy: 0.8734 - val_loss: 3.7043e-04 - val_accuracy: 0.8926
Epoch 2408/5000
66/66 [==============================] - 0s 7ms/step - loss: 3.1107e-04 - accuracy